In [117]:
import uproot, uproot_methods
import uproot_methods.classes.TLorentzVector as TLorentzVector
import matplotlib.pyplot as plt
import numpy as np
import itertools

t = 0.7  # transparency of plots

testRun = True
dR_cut_quarkToJet = 0.40

In [11]:
delphes_hh= uproot.open('../../../MG5_aMC_v2_6_1/ppToHHto4b_14TeV/Events/run_02_decayed_1/tag_1_delphes_events.root')['Delphes']
#b_particles = uproot.tree.TBranchMethods.array(delphes_hh['Particle'])
#l_genD1     = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.D1']).tolist()
#l_genD2     = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.D2']).tolist()
l_genPID    = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.PID']).tolist()
l_genStatus = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.Status']).tolist()
l_genPt     = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.PT']).tolist()
l_genEta    = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.Eta']).tolist()
l_genPhi    = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.Phi']).tolist()
l_genMass   = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.Mass']).tolist()
l_jetPt     = uproot.tree.TBranchMethods.array(delphes_hh['Jet']['Jet.PT']).tolist()
l_jetEta    = uproot.tree.TBranchMethods.array(delphes_hh['Jet']['Jet.Eta']).tolist()
l_jetPhi    = uproot.tree.TBranchMethods.array(delphes_hh['Jet']['Jet.Phi']).tolist()
l_jetMass   = uproot.tree.TBranchMethods.array(delphes_hh['Jet']['Jet.Mass']).tolist()


In [110]:
def returnListOfTruthBQuarkIndicesByDaughters(_D1, _D2, _PID):
    _bQuarkIndices = []
    
    for iParticle in range(0, len(_D1)):
         if _PID[iParticle]==25:
            _daughter1 = _D1[iParticle]
            _daughter2 = _D2[iParticle]
            _daughter1_PID = _PID[daughter1]
            _daughter2_PID = _PID[daughter2]
            #print('Event ',iEvt,'has higgs at position',iParticle,'with daughter1 (',daughter1,
            #    ') of PID',daughter1_PID,'and daughter2 (',daughter2,') of PID',daughter2_PID)
            if abs(_daughter1_PID) == 5 and abs(_daughter2_PID)==5:
                _bQuarkIndices.append(_daughter1)
                _bQuarkIndices.append(_daughter2)
    
    return _bQuarkIndices


def returnListOfTruthBQuarkIndicesByStatus(_status):
    _bQuarkIndices = []

    for iParticle in range(0, len(_status)):
        if _status[iParticle]==23:
            _bQuarkIndices.append(iParticle)

    return _bQuarkIndices

def returnNumberAndListOfJetIndicesPassingCuts(_jetPt, _jetEta, _cut_jetPt, _cut_jetEta):
    _jetIndices = []
    _nJets = 0
    
    for iJet in range(0, len(_jetPt)): 
        if _jetPt[iJet] > _cut_jetPt and abs(_jetEta[iJet]) < _cut_jetEta:
            _jetIndices.append(iJet)
            _nJets += 1
            
    return _nJets, _jetIndices

In [111]:
def getDictOfQuarksMatchedToJets(_quarkIndices, _jetIndices, _genPt, _genEta, _genPhi, _genMass, _jetPt, _jetEta, _jetPhi, _jetMass): 
    _matchedQuarksToJets = {}
    _dictOfJetVectors = {}
    _dictOfQuarkVectors = {}
    
    for iQuark in _quarkIndices:
        tlv_quark = TLorentzVector.PtEtaPhiMassLorentzVector( _genPt[iQuark], _genEta[iQuark], _genPhi[iQuark], _genMass[iQuark])
        if iQuark not in _dictOfQuarkVectors.keys():
            _dictOfQuarkVectors[iQuark] = tlv_quark
            
        for iJet in _jetIndices:
            tlv_jet = TLorentzVector.PtEtaPhiMassLorentzVector( _jetPt[iJet], _jetEta[iJet], _jetPhi[iJet], _jetMass[iJet])
            if iJet not in _dictOfJetVectors.keys():
                _dictOfJetVectors[iJet] = tlv_jet
        
            # skip jets
            if tlv_quark.delta_r( tlv_jet) > dR_cut_quarkToJet:
                continue

            if iQuark not in _matchedQuarksToJets.keys():
                _matchedQuarksToJets.update({iQuark:[iJet]})
            else:
                _matchedQuarksToJets[iQuark].append(iJet)

    return _matchedQuarksToJets, _dictOfJetVectors, _dictOfQuarkVectors



In [135]:
def returnDeltaR(_jetIndexTuple):
    return 0
    

def selectPairsViaMatchingAlgorithm(_jetVectorDict, _matchingAlgorithm):

    _jetPairs = list(itertools.combinations(_jetVectorDict.keys(),2))
    _doubleJetPairs = {}
    
    for pair in _jetPairs:
        _notPair = [x for x in list(_jetPairs) if pair[0] not in x and pair[1] not in x]
        for pairOption in _notPair:
            _sortedPairing = sorted([sorted(x) for x in [pair, pairOption]])
            _sortedTuple = tuple(_sortedPairing[0]+_sortedPairing[1])

            if _sortedTuple not in _doubleJetPairs.keys():
                _doubleJetPairs[_sortedTuple]=0

    #print(len(_doubleJetPairs), len(_jetVectorDict))
    #print(_doubleJetPairs)
    return 0,1

In [136]:
nEventsWithExactly4Jets = 0
nEventsWithAtLeast4Jets = 0
nEventsWith4MatchableJets = 0

for iEvt in range(0,delphes_hh.fEntries):
    # *** 0. Kick-out condition for testing
    if iEvt > 40 and testRun is True:
        continue
    if iEvt%2000==0:
        print("Analyzing event number",iEvt)
    
    # *** 1. Get truth information
    indicesByStatus    = returnListOfTruthBQuarkIndicesByStatus(l_genStatus[iEvt])   
    if len(indicesByStatus) != 4:
        print ("!!! WARNING: iEvt = {0} did not find 4 truth b-quarks. Only found {1} !!!".format(iEvt, len(indicesByStatus)))
        continue
        
    # *** 2. Get jet reco information
    nJets, jetIndices = returnNumberAndListOfJetIndicesPassingCuts(l_jetPt[iEvt], l_jetEta[iEvt], 20.0, 2.5)
    if nJets < 4: continue 
    nEventsWithAtLeast4Jets += 1
    if nJets==4: nEventsWithExactly4Jets += 1 

        
    # *** 3. Do some matching
    matchedQuarksToJets, jetVectorDict, quarkVectorDict = getDictOfQuarksMatchedToJets(indicesByStatus, jetIndices, l_genPt[iEvt], l_genEta[iEvt], l_genPhi[iEvt], l_genMass[iEvt], l_jetPt[iEvt], l_jetEta[iEvt], l_jetPhi[iEvt], l_jetMass[iEvt])
    #print( matchedQuarksToJets, all(len(matchedJets) == 1 for matchedJets in matchedQuarksToJets.values()) )
    if all(len(matchedJets) == 1 for matchedJets in matchedQuarksToJets.values()) and len(matchedQuarksToJets)==4:
        nEventsWith4MatchableJets += 1
    
    
    # *** 4. Pick two jet pairs based on some algorithm
    jetPair1, jetPair2 = selectPairsViaMatchingAlgorithm(jetVectorDict, "minDeltaR")
    
    
print("Number of Events with >= 4 jet:",nEventsWithAtLeast4Jets)
print("Number of Events with == 4 jet:",nEventsWithExactly4Jets)
print("Number of Events with 4 truth-matchable jets:",nEventsWith4MatchableJets)
    


Analyzing event number 0
Number of Events with >= 4 jet: 31
Number of Events with == 4 jet: 9
Number of Events with 4 truth-matchable jets: 12


In [89]:
import itertools
a=[1,2,3,4,5,6,7]
pairs = list(itertools.combinations(a,2))
#list(pairs)
pairPairs = {}
for two in pairs:
    notwo = [x for x in list(pairs) if two[0] not in x and two[1] not in x]
    for pairOption in notwo:
        sortedPairing = sorted([sorted(x) for x in [two, pairOption]])
        sortedTuple = tuple(sortedPairing[0]+sortedPairing[1])

        if sortedTuple not in pairPairs.keys():
            pairPairs[sortedTuple]=0

print(len(pairPairs))
print(pairPairs)
    

105
{(1, 2, 3, 4): 0, (1, 2, 3, 5): 0, (1, 2, 3, 6): 0, (1, 2, 3, 7): 0, (1, 2, 4, 5): 0, (1, 2, 4, 6): 0, (1, 2, 4, 7): 0, (1, 2, 5, 6): 0, (1, 2, 5, 7): 0, (1, 2, 6, 7): 0, (1, 3, 2, 4): 0, (1, 3, 2, 5): 0, (1, 3, 2, 6): 0, (1, 3, 2, 7): 0, (1, 3, 4, 5): 0, (1, 3, 4, 6): 0, (1, 3, 4, 7): 0, (1, 3, 5, 6): 0, (1, 3, 5, 7): 0, (1, 3, 6, 7): 0, (1, 4, 2, 3): 0, (1, 4, 2, 5): 0, (1, 4, 2, 6): 0, (1, 4, 2, 7): 0, (1, 4, 3, 5): 0, (1, 4, 3, 6): 0, (1, 4, 3, 7): 0, (1, 4, 5, 6): 0, (1, 4, 5, 7): 0, (1, 4, 6, 7): 0, (1, 5, 2, 3): 0, (1, 5, 2, 4): 0, (1, 5, 2, 6): 0, (1, 5, 2, 7): 0, (1, 5, 3, 4): 0, (1, 5, 3, 6): 0, (1, 5, 3, 7): 0, (1, 5, 4, 6): 0, (1, 5, 4, 7): 0, (1, 5, 6, 7): 0, (1, 6, 2, 3): 0, (1, 6, 2, 4): 0, (1, 6, 2, 5): 0, (1, 6, 2, 7): 0, (1, 6, 3, 4): 0, (1, 6, 3, 5): 0, (1, 6, 3, 7): 0, (1, 6, 4, 5): 0, (1, 6, 4, 7): 0, (1, 6, 5, 7): 0, (1, 7, 2, 3): 0, (1, 7, 2, 4): 0, (1, 7, 2, 5): 0, (1, 7, 2, 6): 0, (1, 7, 3, 4): 0, (1, 7, 3, 5): 0, (1, 7, 3, 6): 0, (1, 7, 4, 5): 0, (1, 7, 4,

In [126]:
a=[[1, 2], [3, 4]]
b = tuple(a[0]+a[1])
type((b[0],b[1]))

tuple